In [1]:
from ultralytics import YOLO
import cv2
import supervision as sv

In [2]:
model = YOLO('D:/projects/football_analysis/models/best_11n.pt')

In [3]:
cap = cv2.VideoCapture('D:/projects/football_analysis/input_videos/08fd33_4.mp4')
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

In [5]:
detections = []
detection = model.predict(frames[:20], conf=0.1)
detections += detection
detection2 = model.predict(frames[:20], conf=0.1)
detections += detection2


0: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 64.1ms
1: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 64.1ms
2: 384x640 1 goalkeeper, 22 players, 4 referees, 64.1ms
3: 384x640 1 ball, 1 goalkeeper, 22 players, 4 referees, 64.1ms
4: 384x640 1 ball, 1 goalkeeper, 22 players, 3 referees, 64.1ms
5: 384x640 1 goalkeeper, 23 players, 4 referees, 64.1ms
6: 384x640 1 goalkeeper, 23 players, 4 referees, 64.1ms
7: 384x640 1 ball, 1 goalkeeper, 22 players, 4 referees, 64.1ms
8: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 64.1ms
9: 384x640 1 ball, 20 players, 3 referees, 64.1ms
10: 384x640 1 ball, 20 players, 3 referees, 64.1ms
11: 384x640 19 players, 3 referees, 64.1ms
12: 384x640 20 players, 3 referees, 64.1ms
13: 384x640 1 ball, 20 players, 3 referees, 64.1ms
14: 384x640 1 ball, 21 players, 3 referees, 64.1ms
15: 384x640 23 players, 3 referees, 64.1ms
16: 384x640 21 players, 3 referees, 64.1ms
17: 384x640 1 ball, 21 players, 3 referees, 64.1ms
18: 384x640 1 ball, 2

In [6]:
type(detections)

list

In [7]:
len(detections)

40

In [8]:
detections[1].names

{0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}

38 prediction k baad 39 frame pe dikkat aa rhi h

  File "D:\projects\football_analysis\main.py", line 11, in main
    tracks = tracke.get_object_tracks(video_frames, read_from_stub=True, stub_path='stubs/track_stubs.pkl')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\projects\football_analysis\src\trackers\tracker.py", line 60, in get_object_tracks
    tracks["players"][frame_num][track_id] = {"bbox":bbox}
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^
IndexError: list index out of range

In [9]:
model = YOLO('D:/projects/football_analysis/models/best_11n.pt')

In [10]:
cap = cv2.VideoCapture('D:/projects/football_analysis/input_videos/08fd33_4.mp4')
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

In [12]:
len(frames)

750

In [16]:
frames[0].shape

(1080, 1920, 3)

In [17]:
frames[1].shape

(1080, 1920, 3)

In [19]:
batch_size = 20
detections = []
for i in range(0, len(frames), batch_size):
    detection_batch = model.predict(frames[i:i+batch_size], conf=0.1)
    detections += detection_batch


0: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 52.5ms
1: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 52.5ms
2: 384x640 1 goalkeeper, 22 players, 4 referees, 52.5ms
3: 384x640 1 ball, 1 goalkeeper, 22 players, 4 referees, 52.5ms
4: 384x640 1 ball, 1 goalkeeper, 22 players, 3 referees, 52.5ms
5: 384x640 1 goalkeeper, 23 players, 4 referees, 52.5ms
6: 384x640 1 goalkeeper, 23 players, 4 referees, 52.5ms
7: 384x640 1 ball, 1 goalkeeper, 22 players, 4 referees, 52.5ms
8: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 52.5ms
9: 384x640 1 ball, 20 players, 3 referees, 52.5ms
10: 384x640 1 ball, 20 players, 3 referees, 52.5ms
11: 384x640 19 players, 3 referees, 52.5ms
12: 384x640 20 players, 3 referees, 52.5ms
13: 384x640 1 ball, 20 players, 3 referees, 52.5ms
14: 384x640 1 ball, 21 players, 3 referees, 52.5ms
15: 384x640 23 players, 3 referees, 52.5ms
16: 384x640 21 players, 3 referees, 52.5ms
17: 384x640 1 ball, 21 players, 3 referees, 52.5ms
18: 384x640 1 ball, 2

In [21]:
len(detections)

750

In [23]:
type(detections[0])

ultralytics.engine.results.Results

In [81]:
tracks = {
    "players":[],
    "referees":[],
    "ball":[]
}

In [33]:
tracker = sv.ByteTrack()

In [ ]:
for frame_num, detection in enumerate(detections):
    cls_names = detection.names
    cls_names_inv = {v:k for k, v in cls_names.items()}

    detection_supervision = sv.Detections.from_ultralytics(detection)
    
    for obj_idx, class_id in enumerate(detection_supervision.class_id):
        if cls_names[class_id] == "goalkeeper":
            detection_supervision.class_id[obj_idx] = cls_names_inv["player"]

In [28]:
dd = sv.Detections.from_ultralytics(detections[0])

In [29]:
type(dd)

supervision.detection.core.Detections

In [32]:
dd.class_id

array([2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 0, 2, 2, 3])

In [35]:
dd_with_track = tracker.update_with_detections(dd)

In [36]:
type(dd_with_track)

supervision.detection.core.Detections

In [39]:
dd.tracker_id

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, -1, -1, -1, -1])

In [38]:
dd_with_track.tracker_id

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])

One object

In [42]:
dd_with_track[0]

Detections(xyxy=array([[     586.47,      589.19,      630.59,      673.91]], dtype=float32), mask=None, confidence=array([    0.91911], dtype=float32), class_id=array([2]), tracker_id=array([1]), data={'class_name': array(['player'], dtype='<U10')}, metadata={})

In [83]:
dd_with_track[0].xyxy.tolist()[0]

[586.4676513671875, 589.185302734375, 630.5856323242188, 673.9056396484375]

In [73]:
dd_with_track[0].class_id

array([2])

In [74]:
dd_with_track[0].tracker_id

array([1])

In [80]:
tracks['players']

1

In [82]:
def add_track_data(tracks, category, frame_num, track_id, bbox):
    while len(tracks[category]) <= frame_num:
        tracks[category].append({})
    tracks[category][frame_num][track_id] = {"bbox": bbox}

In [84]:
add_track_data(tracks, "players", 0, 0, dd_with_track[0].xyxy.tolist()[0])

In [90]:
tracks["players"][0][0]

{'bbox': [586.4676513671875,
  589.185302734375,
  630.5856323242188,
  673.9056396484375]}